<a href="https://colab.research.google.com/github/Sammy-Reeves/Issue-2-electric-bagaloo/blob/main/Week_2_NDIs/1_Calculate_NDIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate a normalised difference index in Google Earth Engine

## Log in to Google Earth Engine

Same as last week, we must log in to Google Earth Engine. Note that we are importing an additional library this week - `geemap`'s 'chart' library.

In [1]:
import ee
import geemap
import geemap.chart as chart
import time

ee.Authenticate()  # Trigger the authentication flow.
ee.Initialize(project='ee-sammyreeveswork')    # Change to your own default project name.

## Search for a scene

Again, as last week, we will outline some search parameters. This week, we will look at Quelcayya Ice Cap (QIC), in Peru, with the aim of extracting an outline of the ice cap.

One note that I am considering this week that we didn't last week is the _time period_ over which we are searching. When extracting glacier boundaries, we want to do this at the end of the ablation season (i.e. the summer), so that we are not misclassifying snow. This is generally considered to be August-September in the Northern hemisphere. QIC is a little different as it sits in the tropics - instead, we choose the end of the dry season, which is nominally ~September. We will search for the least cloudy image between mid August and end September.

When you're doing your project, you will need to consider how the timing of the scene impacts your own interpretation, especially if you're comparing inter-annual scenes. For instance, if you're trying to assess deforestation in a decidious forest, there's no point comparing a summer scene to an autumn scene!

In [2]:

# Location - editable
latitude = 54.3   # Degrees of latitude
longitude = -0.6  # Degrees of longitude
size = 16000  # Size of AOI, in metres
location_name = 'yorkshireforest2'  # recognisable name, to create a useful file name

# Dates - editable
date_start = '2019-05-01'
date_end = '2023-09-30'

# Set up location geometry
point = ee.Geometry.Point(longitude, latitude)  # Create a point
region = point.buffer(size/2).bounds()  # Buffer the point to a 2D shape


Let's check we've got a good area. I had to increase the `size` parameter from 10 km to 15 km relative to last week's example to properly capture the full extent of the ice cap. You can explore increasing this as much as you like. The 'swath width' (width of the scene) of Landsat is 185 km, so we can go much larger. However, I wouldn't recommend it - your file sizes and processing times may become unweildy. Consider what is a reasonable region of interest to investigate.

In [3]:
Map = geemap.Map()  # Create an empty Map
Map.addLayer(region, {}, "Search Region")  # Add our AOI
Map.centerObject(region, zoom=12)  # Centre our map on the region of interest
Map

Map(center=[54.29999257137368, -0.5997814549419791], controls=(WidgetControl(options=['position', 'transparent…

As with last week, we can find the least cloudy image in our search window:

In [4]:
# Get Landsat 8 image collection
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

# Filter to desired region and date bounds
landsat8_collection = landsat8_collection.filterBounds(region)
landsat8_collection = landsat8_collection.filterDate(date_start, date_end)

# Get the least cloudy image in the collection, and clip it to our search region
image = landsat8_collection.sort('CLOUD_COVER').first()
image = image.clip(region)

# Print out date, for reference
date = image.get('DATE_ACQUIRED').getInfo()
print(f'Selected image on date {date}')

image

Selected image on date 2019-08-26


And below, we visualise it to make sure it's good quality. Note I have increased the `max_reflectance` value to 0.8, so that we can see features of the snow and ice. This has the side effect of making the bare ground look much darker.

In [7]:

Map = geemap.Map()  # Create empty map

max_reflectance = 0.10 # Set the upper limit of reflectance to visualise.
                       # Play with this value (between 0-1) to see what it
                       # does. It will need to be higher for snowy/icy
                       # scenes

visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'Colour Composite Image')

Map.centerObject(region, zoom=12)
Map

Map(center=[54.29999257137368, -0.5997814549419791], controls=(WidgetControl(options=['position', 'transparent…

# Calculating a normalised difference index

Recall from the lecture slides last week that a normalised difference index takes the form

$$NDI = \frac{r_{high} - r_{low}}{r_{high} + r_{low}}$$

Where $r_{high}$ is a band that has characteristically high reflectance for our surface of interest, and $r_{low}$ has characteristically low reflectance. From this principle, there are a number of useful normalised difference indices we can use (and more I'm sure you can find):

| Name | Equation |
| ---- | -------- |
| Normalised Difference Vegetation Index | $$NDVI = \frac{NIR_{B5} - Red_{B4}}{NIR_{B5} + Red_{B4}}$$ |
| Normalised Difference Snow Index | $$NDSI = \frac{Green_{B3} - SWIR_{B6}}{Green_{B3} + SWIR_{B6}}$$ |
| Normalised Difference Water Index | $$NDWI = \frac{Green_{B3} - NIR_{B5}}{Green_{B3} + NIR_{B5}}$$ |
| Normalised Difference Built-up Index | $$NDBI = \frac{SWIR_{B6} - NIR_{B5}}{SWIR_{B6} + NIR_{B5}}$$ |
| Normalised Difference Water Index (ice) | $$NDWI_{ice} = \frac{Red_{B4} - Blue_{B2}}{Red_{B4} + Blue_{B2}}$$ |
| Normalised Burn Ratio | $$NBR = \frac{NIR_{B05} - SWIR_{B6}}{NIR_{B5} + SWIR_{B6}}$$ |

I have included the relevant Landsat 8/9 bands as subscript in the equations - note this won't be the same if you start using, e.g., Landsat 1-7 or Sentinel-2 in the future.

Google Earth Engine has a [fantastically useful function](https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference) to help us simplify this: `ee.Image.normalizedDifference(['r_high', 'r_low'])` (note the American spelling of 'normalized'). This takes the name of two bands - $r_{high}$ and $r_{low}$ - and produces an NDI product.

All we have to do is make sure we know the correct names of the bands. They are also listed, alongside detailed information, [here](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_TOA#bands). However, we can also extract them from the `image` object as follows (:

In [9]:
image.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'QA_PIXEL',
 'QA_RADSAT',
 'SAA',
 'SZA',
 'VAA',
 'VZA']

Using this information, we can calculate an NDI. As we're currently looking at an ice cap, it makes sense to calculate NDSI for now. Consulting the table above, this means using the green band (band 3) and the short-wave infrared (SWIR) band (band 6). Let's do this:

In [10]:

type_of_ndi = 'NDVI'  # type of NDI calculated, for filename purposes
r_high = 'B5'  # relevant band name for r_high
r_low = 'B4'   # relevant band name for r_low

ndi = image.normalizedDifference([r_high, r_low]).rename(type_of_ndi)

ndi

Now, we can visualise the product. Note that I have visualised both the NDSI and the colour image as layers - click the 🔧 symbol in the top right, and then the ☰ button, to open a menu where you can toggle the NDSI layer on and off to compare the two.

In [11]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
visParams = {'bands': [type_of_ndi], 'min': -1, 'max': 1, 'palette': ['red', 'white', 'blue']}
Map.addLayer(ndi, visParams, type_of_ndi)

Map.centerObject(region, zoom=12)
Map


Map(center=[54.29999257137368, -0.5997814549419791], controls=(WidgetControl(options=['position', 'transparent…

Have a zoom around and see what you think. Overall, this is a good product - there's clear boundaries and not much confusion over shaded areas, etc. However, it's clear that the NDSI is also picking up lakes/water as ice. This is an established weakness of NDSI.

> **Task:**
>
> 1. Go back and calculate NDWI instead of NDSI. Is NDWI better at differentiating water and ice?
>
> 2. Based on the bands used, why do you think this is?
>
> 3. Have an explore of other NDIs as well (e.g. NDVI).

After you have explored this, return to calculating the NDSI and ensure the cells have run so that the `ndi` variable is storing the NDSI.

# Thresholding

One common use of NDIs is to _threshold_ the data, extracting a binary mask of identified features. For instance, we might say that anything with an NDVI value above 0.4 is vegetation. In order to do this, we need to choose a sensible threshold.

A good way to begin manually identifying a threshold is to construct a histogram of the computed NDI values in the scene. The short script below samples 10,000 pixels within the NDI scene and presents them as a histogram. Take a look at the code and then run it to see the histogram:

In [30]:

# Sample 10,000 pixels within the NDI image
sample_pixels = ndi.sample(region, numPixels=10000)

# Set labels for the graph
labels = {
    "title": 'Distribution of NDI values within image',
    "xlabel": f'{type_of_ndi} values',
    "ylabel": 'Pixel count',
}

# Construct the histogram
chart.feature_histogram(sample_pixels, type_of_ndi, **labels)

We can see that the data has a _strongly bimodal_ distribution, with lots of land surface with NDSI values <0.1 and glaciated surface with NDI values >0.8. As a result, our choice of threshold here is not too important - anything between 0 and 0.8 would probably result in a reasonable ice surface mask. For other environments, the distinction might be harder to call, and you might need to explore different thresholds and see what they look like.

For our purposes, let's choose 0.8 to minimise the amount of _false positive_ pixels:

In [49]:
# manually set a threshold
threshold = 0.645

# threshold the image to where greater than threshold.
# note that you could change from greater than to less than by
# changing 'gt' to 'lt'.
ndi_threshold = ndi.gt(threshold)

# 'Mask' the data, showing only regions beyond the threshold.
ndi_threshold = ndi_threshold.updateMask(ndi_threshold.neq(0))


The above cell creates a new image where values greater than the threshold are set to 1, and values beneath the threshold are set to 0 (and in the background, values of zero are 'masked', so that they are transparent in future visualisations).

We can visualise this data semi-transparent over the original data to see how we've done:

In [50]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.40
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
visParams = {'bands': [type_of_ndi], 'palette': ['white', 'cyan'], 'opacity': 0.3}
Map.addLayer(ndi_threshold, visParams, type_of_ndi)

Map.centerObject(region, zoom=12)
Map


Map(center=[54.29999257137368, -0.5997814549419791], controls=(WidgetControl(options=['position', 'transparent…

Not bad.
<!-- Now that we have a clear mask of snow vs bedrock, we could even begin to calculate quantiative statistics, such as the total area that is snow. We will calculate this in km3: -->

 We could choose to export this image for visualisation, but it is better I think to export this as _vector_ data rather than _raster_ data (recall last year's course which covered the two).

We can do this simply with Earth Engine's `.reduceToVectors()` tool. Alongside this, I've written some code to also extract the _area_ of each vector polygon. You'll see what this does in the next step.

In [1]:

# Calculate the area of each pixel in km2, and add it as a band to the NDI image
ndi_and_area = ndi_threshold.addBands(ndi_threshold.multiply(30*30).divide(1e6))

# Use the `reduceToVectors()` function to produce vectors, also calculating
# the total area using the `sum()` function to sum the pixel areas.
vectors = ndi_and_area.reduceToVectors(
  scale=30,
  geometryType = 'polygon',
  eightConnected = False,
  reducer = ee.Reducer.sum(),
)

# Extra line to rename the area column to `area_km2`.
vectors = vectors.map(lambda feature: feature.set('area_km2', feature.get('sum')).set('sum', None).set('label', None))


NameError: name 'ndi_threshold' is not defined

Let's visualise the vector layer over the top of the image:

In [ ]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
Map.addLayer(vectors, {'color': 'cyan'}, "Identified Region")  # Add our AOI

Map.centerObject(region, zoom=12)
Map


As we have calculated the area, we could even get quantative statistics:

In [ ]:

# Get total area of initial search region
aoi_area_km2 = region.area(maxError=1).getInfo() / 1e6

# Get total area of all vectors
vector_area_km2 = vectors.aggregate_sum('area_km2').getInfo()

# Print the results
print(f'Total scene area: {aoi_area_km2:.2f} km2')
print(f'Total classified area: {vector_area_km2:.2f} km2')


However, as you might have noticed in the map, we have a bit of noise going on!

We can fix this by filtering the features only to those of a sufficiently large area. It would be nice to just have the ice cap, so let's set a fairly large theshold area of 0.5 km<sup>2</sup>.

In [ ]:
# Set filter threshold value, in km2
filter_threshold_km2 = 0.5

# Filter polygons smaller than this chosen threshold
vectors_filtered = vectors.filter(ee.Filter.gt('area_km2', filter_threshold_km2))

vectors_filtered

Let's visualise this again...

In [ ]:

Map = geemap.Map() # Create empty map

# Display colour image
max_reflectance = 0.80
visParams = {'bands': ['B4', 'B3', 'B2'], 'max': max_reflectance}
Map.addLayer(image, visParams, 'True Colour')

# Display NDI
Map.addLayer(vectors_filtered, {'color': 'cyan'}, "Identified Region")  # Add our AOI

Map.centerObject(region, zoom=12)
Map


Beautiful! We can finally calculate the area of the ice cap:

In [ ]:

# Get total area of initial search region
aoi_area_km2 = region.area(maxError=1).getInfo() / 1e6

# Get total area the filtered vectors
vector_area_km2 = vectors_filtered.aggregate_sum('area_km2').getInfo()

# Print the results
print(f'Total scene area: {aoi_area_km2:.2f} km2')
print(f'Total classified area: {vector_area_km2:.2f} km2')


## Download data

Let's set the Google Drive folder we want to export to. This should be the same as last week:

In [ ]:
# You can edit this variables
folder = 'scires_project_2A'


First, let's download the initial scene:

In [ ]:
# Construct the filename automatically
date_string = image.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_string + '_image'

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': image.select(['B4', 'B3', 'B2', 'B5', 'B6']),
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


Now, let's download the NDI value we produced

In [ ]:
# Construct the filename automatically
date_string = image.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_string + '_' + type_of_ndi

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.tif\n')

# Export the image, specifying scale and region.
task = ee.batch.Export.image.toDrive(**{
    'image': ndi,
    'description': filename,
    'folder': folder,
    'scale': 30,
    'region': region.getInfo()['coordinates']
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Image is exported to your Drive.')


Finally, let's download the vector data. We could download this as either a shapefile or a KML file (which is Google's preferred format for Google Earth and other software, but we can also use it in QGIS). Note in the export function I have currently commented out the `'KML'` line and kept in the `'SHP'` line. You can swap these around if you would rather have a KML.

As you may remember from last year, shapefiles are multi-file and slightly unweildy, but they are also editable in QGIS (so you could delete smaller polygons if you like) so we'll use them for now. If you aren't bothered about the possibility of needing to this, ML files aren't as feature-rich but are only one file so are easy to manage.

In [ ]:
# Construct the filename automatically
date_string = image.get('DATE_ACQUIRED').getInfo()
filename = location_name + '_' + date_string + '_' + type_of_ndi + '_theshold_' + str(max_reflectance)

# Print out filename for reference
print("The image will be saved to your Google Drive at:\n" + folder + '/' + filename + '.kml\n')

# Export the featureCollection, specifying scale and region.
task = ee.batch.Export.table.toDrive(**{
  'collection': vectors_filtered,
  'description': filename,
  'folder': folder,
  # 'fileFormat': 'KML',
  'fileFormat': 'SHP',
})
task.start()

while task.active():
  print('Task processing ongoing... (id: {}).'.format(task.id))
  time.sleep(5)

print('Finished processing. Vector file is exported to your Drive.')


Good job! Download these files from your Google Drive to a sensible place in your local storage, and then move on to the next notebook to visualise these in QGIS.